# Задание 1

Дана зависимость энтальпии от температуры:

Определить значения энтальпии при изменении $Т$ от $300$ до $1000$ с шагом $50$ К, используя:

1. Кубический сплайн;
2. Линейную аппроксимацию

In [1]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import leastsq

In [2]:
def linear(x, params):
    a, b = params
    return a + b * x

In [3]:
def residuals(params, x, y, func):
    return y - func(x, params)

In [6]:
t = np.array([300, 400, 500, 600, 700, 800, 900, 1000])
dh = np.array([29.62, 21.88, 15.52, 10.38, 6.40, 3.35, 1.13, 0.21])

In [7]:
t_new = np.arange(300, 1050, 50)

In [9]:
cubic = interp1d(t, dh, kind='cubic')
dh_cubic = cubic(t_new)
dh_cubic

array([29.62      , 25.56507775, 21.88      , 18.53992225, 15.52      ,
       12.80023325, 10.38      ,  8.25664474,  6.4       ,  4.7744378 ,
        3.35      ,  2.11560407,  1.13      ,  0.46939593,  0.21      ])

In [10]:
lparams, *_ = leastsq(residuals, x0=(.5, .5), args=(t, dh, linear))
dh_linear = linear(t_new, lparams)
dh_linear

array([25.64916667, 23.56517857, 21.48119048, 19.39720238, 17.31321429,
       15.22922619, 13.14523809, 11.06125   ,  8.9772619 ,  6.89327381,
        4.80928571,  2.72529762,  0.64130952, -1.44267857, -3.52666667])

# Задание 2

Используя функцию `scipy.integrate.quad`, вычислите значение энтропии воды при ее нагревании от $400$ до $500$ К по формуле:
	
$ \Delta S = \eta \cdot \int\limits_{400}^{500} \dfrac{C_v \cdot dT}{T} $
	
Количество молей $\eta = 3$; значение теплоемкости $C_v = 35.0$ Дж / (моль $\cdot$ К).

In [11]:
from scipy.integrate import quad

In [12]:
def func(t, cv, eta):
    return cv * eta / t

In [14]:
a, b = 400, 500
cv, eta = 35, 3

In [15]:
res = quad(func, a, b, args=(cv, eta))
res

(23.430072887992026, 2.6012606388894386e-13)

# Задание 3

Дана схема химических превращений:

$
\left\{
	\begin{aligned}
		\dfrac{dC_A}{dt} &= -k_1 \cdot C_A -k_2 \cdot C_A + k_3 \cdot C_C \\
		\dfrac{dC_B}{dt} &=  k_1 \cdot C_A  \\
		\dfrac{dC_C}{dt} &=  k_2 \cdot C_A - k_3 \cdot C_C \\
	\end{aligned}
\right.
$

$C_{A_0} = 0.7$ $\left(\text{моль/л}\right)$; $k_1 = 0.21 \left(c^{-1}\right)$;

$C_{B_0} = 0.0 \left(\text{моль/л}\right)$; $k_2 = 0.12 \left(c^{-1}\right)$;

$C_{C_0} = 0.0 \left(\text{моль/л}\right); k_3 = 0.18 \left(c^{-1}\right)$.

Решите систему дифференциальных уравнений изменения концентраций веществ по времени на отрезке $\left[0;10\right]$ с шагом $h=1$.

In [16]:
from scipy.integrate import solve_ivp

In [17]:
c0 = .7, .0, .0
k = .21, .12, .18
t0, tf, h = 0, 10, 1
t = np.arange(t0, tf+h, h)

In [18]:
def foo(t, c, k):
    return [
        -k[0] * c[0] - k[1] * c[0] + k[2] * c[2],
        k[0] * c[0],
        k[1] * c[0] - k[2] * c[2]
    ]

In [20]:
solution = solve_ivp(
    foo, (t0, tf), c0, dense_output=True, args=(k,)
)
ca, cb, cc = solution.sol(t)

In [22]:
for a, b, c in zip(ca, cb, cc):
    print(f'{a:>8.4f} {b:>8.4f} {c:>8.4f}')

  0.7000   0.0000   0.0000
  0.5090   0.1256   0.0654
  0.3792   0.2180   0.1027
  0.2901   0.2878   0.1221
  0.2281   0.3418   0.1301
  0.1841   0.3848   0.1311
  0.1522   0.4199   0.1279
  0.1286   0.4493   0.1221
  0.1106   0.4743   0.1151
  0.0966   0.4960   0.1074
  0.0853   0.5151   0.0997
